In [1]:
import numpy as np
from scipy import linalg
#linalg only used for getting the inverse of the matrices

In [2]:
def is_int(num,tol=0.01):
    #checks if input number is an 'integer' within the specified tolerance
    #useful for enforcing integer solutions of matrix equations
    #can be less picky than the native float.isinteger() method,
    #    which can have some trouble with roundoff-error integers
    temp = round(num,0)
    
    if np.abs(temp-num) <= tol:
        return True
    else:
        return False

In [3]:
#LU decomposition...

def LU_step(A,step):
    #this function is used for getting the various Ln's in the process of
    #    LU decomposition. It takes a matrix argument and the "step" of the
    #    process that is being performed.
    size = np.shape(A)[0]
    L = np.zeros((size,size))
    for i in range(size):
        for j in range(size):
            if (i==step) and (j==step):
                L[i][j] = 1
            elif i==j:
                L[i][j] = A[step][step]
            elif (j==step) and (i>step):
                L[i][j] = -1*A[i][step]
                
    L = (1/A[step][step])*L
    return(L)


def my_LU_decomp(A):
    #starting with an input matrix 'A'
    #for the purpose of this assignment, the program is designed to work with either a 3x3 or 8x8 matrix...
    A = np.array(A)
    size = np.shape(A)[0]
    
    #constructing L0:
    L0 = LU_step(A,0)
    B = np.dot(L0,A)
    
    #etc...
    L1 = LU_step(B,1)
    C = np.dot(L1,B)
    
    L2 = LU_step(C,2)
    D = np.dot(L2,C)
    
    #we stop here if A is a 3x3...
    if size==3:
        U = L2 @ L1 @ L0 @ A
        #using scipy linalg inverse method here...
        L = linalg.inv(L0) @ linalg.inv(L1) @ linalg.inv(L2)
        return(L,U)
    
    #we keep going if A is 8x8:
    L3 = LU_step(D,3)
    E = np.dot(L3,D)
    
    L4 = LU_step(E,4)
    F = np.dot(L2,E)
    
    L5 = LU_step(F,5)
    G = np.dot(L5,F)
    
    L6 = LU_step(G,6)
    H = np.dot(L6,G)
    
    L7 = LU_step(H,7)
    I = np.dot(L7,H)
    
    #and return the respective L and U matrices.
    U = L7 @ L6 @ L5 @ L4 @ L3 @ L2 @ L1 @ L0 @ A
    L = linalg.inv(L0) @ linalg.inv(L1) @ linalg.inv(L2) @ linalg.inv(L3) @ \
        linalg.inv(L4) @ linalg.inv(L5) @ linalg.inv(L6) @ linalg.inv(L7)
    return(L,U)
    

In [4]:
# solving the matrices given an L and U matrix from the LU decomposition
def my_LU_solve(L,U,v0,i):
    size = np.shape(L)[0]
    v = i*v0
    y = np.zeros(np.shape(v))
    x = np.zeros(np.shape(v))
    #implements back/forward substitution from (6.36) and (6.35)
    #first we solve Ly=v
    for e in range(size):
        y[e] = (v[e] - np.sum(L[e][:e]*y[:e]))/(L[e][e])
    #now we solve for Ux = y
    for e in range((size-1),-1,-1):
        x[e] = (y[e] - np.sum(U[e][e:]*x[e:]))/(U[e][e])
    
    #complete solution vector (appends the last element)
    x = np.append(x,i)
    return(x)

        
        

# Part a: simple matrix

In [5]:
#matrix for part A
A = [[6,0,0], #eqn balancing Hydrogen
    [0,2,-2], #eqn balancing Oxygen
     [2,0,-1]] #eqn balancing Carbon

v0 = np.array([2,1,0]) #vector in d
            # will try various solutions with difference d's (implemented by v = d*v0) until integer a,b,c's are found

In [6]:
#now we find integer solutions for the matrix:
l,u = my_LU_decomp(A)

#initial guess of d = 1:
a,b,c,d = my_LU_solve(l,u,v0,1)

while ((not(is_int(a))) or (not(is_int(b))) or (not(is_int(c)))):
    d += 1
    a,b,c,d = my_LU_solve(l,u,v0,d)

print("Final Solution vector: ")
print("a = ",a)
print("b = ",b)
print("c = ",c)
print("d = ",d)

Final Solution vector: 
a =  2.0
b =  7.0
c =  4.0
d =  6.0


# Part b, a more complicated system

In [7]:
B = [[4,2,0,0,0,0,0,-2],        #K
    [1,0,0,-2,0,0,0,0],         #Fe
    [6,0,1,-3,-3,0,0,-1],       #S
    [6,0,0,0,0,-1,0,0],         #C
    [0,0,2,0,0,0,-2,0],         #H
    [6,0,0,0,0,0,0,0],          #N
    [0,7,4,-12,-12,-2,-1,-4],   #O
    [0,2,0,0,-2,0,0,0]]         #Cr

v0 = np.array([1,0,0,0,0,1,3,0])

In [8]:
#now we find integer solutions for the matrix:
l,u = my_LU_decomp(B)

#initial guess of i = 1:
a,b,c,d,e,f,g,h,i = my_LU_solve(l,u,v0,1)

while ((not(is_int(a))) or (not(is_int(b))) or (not(is_int(c)))
      or (not(is_int(d))) or (not(is_int(e))) or (not(is_int(f)))
      or (not(is_int(g))) or (not(is_int(h)))):
    i += 1
    a,b,c,d,e,f,g,h,i = my_LU_solve(l,u,v0,i)

print("Final Solution vector: ")
print("a = ",int(a))
print("b = ",int(b))
print("c = ",int(c))
print("d = ",int(d))
print("e = ",int(e))
print("f = ",int(f))
print("g = ",int(g))
print("h = ",int(h))
print("i = ",int(i))

Final Solution vector: 
a =  11
b =  3
c =  -626
d =  5
e =  -187
f =  71
g =  -626
h =  -9
i =  72
